In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


/tmp/ipykernel_39732/3514129619.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

# Functions

In [4]:
ws_hostname = "127.0.1.1"
ws_hostname = "172.27.0.64"


street_field  = "streetName"
housenbr_field = "houseNumber"
postcode_field = "postCode"
city_field  =    "city"
country_field =  "country"
addr_key_field = "addrKey"

# ws_hostname = "192.168.1.3"

In [5]:

sample_size = None# 1000
def get_addresses(addresses_filename):
    addresses = pd.read_csv(addresses_filename,  
                            usecols = lambda x: x in [addr_key_field, "addr_key",
                                       country_field, 
                                       postcode_field, 
                                       city_field, 
                                       street_field, 
                                       housenbr_field,
                                       country_field], 
                           dtype={postcode_field: str, housenbr_field: str, country_field: str})
    
    
    addresses = addresses.rename(columns={"addr_key": addr_key_field})
    if country_field in addresses: 
        addresses[country_field] =addresses[country_field].fillna("Belgique")
    else: 
        addresses[country_field] = "Belgique"
    #addresses = addresses.rename(columns={"index":addr_key_field})
    
    addresses = addresses[addresses[street_field].notnull() & addresses[city_field].notnull()] # & addresses[country_field].isnull() ]
    
    addresses[postcode_field] = addresses[postcode_field].astype(str)
    
    if sample_size: 
        addresses = addresses.sample(sample_size, random_state=0)
    
    return addresses#.drop(country_field, axis=1)

In [79]:
def call_ws(addr_data, check_result=True, structured_osm=False, with_rejected=False, mode="long"): #lg = "en,fr,nl"
    t = datetime.now()
    
    try: 
        r = http.request(
        'GET',
        f'http://{ws_hostname}:5000//REST/nominatimWrapper/v0.1/search',
        fields= { 
             street_field: addr_data[street_field],
             housenbr_field: addr_data[housenbr_field],
             city_field: addr_data[city_field],
             postcode_field: addr_data[postcode_field],
             country_field: addr_data[country_field],
             addr_key_field: addr_data[addr_key_field] if addr_key_field in addr_data else None  ,
             "checkResult" : "yes" if check_result else "no",
             "structOsm" : "yes" if structured_osm else "no",
            "withRejected": "yes" if with_rejected else "no",
            "mode": mode

        })
        

    except Exception as e:
        print("Exception !")
        print(addr_data)
        print(e)
        raise e
        
    if r.status == 204:
        print("No result!")
        print(addr_data)
        print(r.data)
        return
    elif r.status == 200:
        try:
            res = json.loads(r.data.decode('utf-8'))
            res["time"] = (datetime.now() - t).total_seconds()
        except ValueError as ve:

            print("Cannot decode result:")
            print(ve)
            print(r.data.decode('utf-8'))
            return r.data
        return res
    else: 
        print(f"Unknown return code: {r.status} ")
        print(r.data)



In [7]:
def split_columns(addresses_batch):
    output = pd.DataFrame(index=addresses_batch.index, dtype=str)

    for f in addresses_batch:
#        print(f)
        grp= addresses_batch[f].apply(lambda r : pd.Series(r, dtype=str) if pd.notnull(r) else pd.Series(dtype=str)) 
        grp = pd.concat({f: grp}, names=['L0', 'L1'], axis=1)
        
        output = pd.concat([output, grp], axis=1)
    # display(output)
    output.columns = pd.MultiIndex.from_tuples(output.columns)
    return output  

In [8]:
def format_ws_res(addresses):
    # assert addresses.json.apply(lambda js: "match" in js).all()
    assert addresses.json.apply(lambda js: len(js["match"])==1 if js is not None and "match" in js else True).all()
    addresses = addresses.json.apply(lambda js: pd.Series(js["match"][0]) if js is not None and "match" in js else pd.Series(dtype=object))
    
    addresses =  split_columns(addresses)
    return addresses

In [9]:
def call_ws_batch(addr_data, mode="geo", with_rejected=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    file_data = addr_data.to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/REST/nominatimWrapper/v0.1/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "withRejected": "yes" if with_rejected else "no",
        "checkResult" : "yes" if check_result else "no",
        "structOsm"   : "yes" if structured_osm else "no",
        #"extra_house_nbr": "no"
    })
    
    try:
        
        res_json = json.loads(r.data.decode('utf-8'))
        if "match" in res_json:
            match =  split_columns(pd.DataFrame(res_json["match"]))
            rejected= split_columns(pd.DataFrame(res_json["rejected"])) if "rejected" in res_json and len(res_json["rejected"])>0 else pd.DataFrame()
            
        else :
            match =  split_columns(pd.DataFrame(res_json))
            rejected=None
    except ValueError as ve:
        
        print("Cannot decode result:")
        print(ve)
        print(r.data.decode('utf-8'))
     
        return r.data
    
    if rejected is not None:
        return match, rejected
    
    else :
        return match
    


In [10]:
# def expand_batch_address(addresses):
#     for field in [street_field, housenbr_field, postcode_field, city_field]:
#         addresses["addrOut"+field]= addresses.address.apply(lambda d: d[field] if d is not None and field in d else "")
#     return 

# Calls

## Single address calls

In [105]:
res=call_ws({
    addr_key_field: 5,
    street_field:   "Avenue Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgique"}, check_result=False, structured_osm=False, with_rejected=True, mode="geo")
res

{'match': [{'input': {'addrKey': '5'},
   'nominatim': {'lat': '50.8358677', 'lon': '4.3385087', 'placeRank': 30},
   'work': {'method': 'fast'}}],
 'rejected': [{'nominatim': {'lat': '50.8358216',
    'lon': '4.3386884',
    'placeRank': 30},
   'work': {'method': 'fast', 'rejectReason': 'tail', 'distToMatch': 0.014}}],
 'time': 0.093701}

In [12]:
res.keys()

dict_keys(['match', 'rejected', 'time'])

In [99]:
res=call_ws({street_field:   "Avenue Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgique"}, check_result=False, structured_osm=False, with_rejected=True)
res

{'match': [{'work': {'method': 'fast',
    'transformedAddress': 'Avenue Fonsny, 20, 1060 Saint-Gilles, Belgique'},
   'nominatim': {'displayName': '20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
    'placeId': 182128,
    'lat': '50.8358677',
    'lon': '4.3385087',
    'placeRank': 30,
    'houseNumber': '20',
    'road': 'Avenue Fonsny - Fonsnylaan',
    'town': 'Saint-Gilles - Sint-Gillis',
    'county': 'Brussel-Hoofdstad - Bruxelles-Capitale',
    'region': 'Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest',
    'postcode': '1060',
    'country': 'België / Belgique / Belgien',
    'countryCode': 'be'},
   'output': {'streetName': 'Avenue Fonsny - Fonsnylaan',
    'city': 'Saint-Gilles - Sint-Gillis',
    'houseNumber': '20',
    'country': 'België / Belgique / Belgien',
    'postCode': '1060',
    'inHouseNumber': '20

## Batch calls (row by row)

In [14]:
addresses = pd.concat([
   get_addresses("data/address_restobe.csv.gz"),
   get_addresses("data/address_best.csv.gz"),
   get_addresses("data/address_cbe.csv.gz")
])
addresses = addresses.reset_index(drop=True)
addresses[addr_key_field] = addresses.index.astype(str)
addresses

,addrKey,streetName,houseNumber,city,postCode,country
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique
1,1,Rue de la Halle,11,NAMUR,5000,Belgique
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique
3,3,rue du Centre,200,WAIMES,4950,Belgique
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique
...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique


In [15]:
# addresses = addresses[addresses.addr_key.isin([ "2078829"])]#"1622",
# addresses.dtypes


### Simple way

In [16]:
# addresses = addresses[addresses.streetName.str.lower().str.contains("steenweg op antwerpen")]

# # addresses = addresses.loc[[100,112,118,142,155,179,198]]
# # addresses = addresses.loc[[ 94,  100,  198, ]]
# addresses = addresses.loc[0:100]
# addresses

In [113]:
addresses_seq = addresses.copy()

t = datetime.now()
addresses_seq["json"] = addresses_seq.fillna("").progress_apply(call_ws, check_result=False, structured_osm=False, 
                                                                mode="short",axis=1)
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_seq.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 128.78 seconds, 7.77 it/s
# Fastmode:     68.80 seconds, 14.54 it/s

#Resto dataset: 
# Normal mode: 145.73 seconds, 6.86 it/s
# Fast mode:    82.99 seconds, 12.05 it/s

# Best dataset:
# Normal mode: 108.53 seconds, 9.21 it/s
# Fast mode: 37.44 seconds, 26.71 it/s

addresses_seq

  0%|          | 0/3000 [00:00<?, ?it/s]

No result!
addrKey                      784
streetName     route de Balmoral
houseNumber                   35
city                SART-LEZ-SPA
postCode                    4845
country                 Belgique
Name: 784, dtype: object
b''
157.00 seconds, 19.11 it/s


,addrKey,streetName,houseNumber,city,postCode,country,json
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique,"{'match': [{'input': {'addrKey': '0'}, 'nomina..."
1,1,Rue de la Halle,11,NAMUR,5000,Belgique,"{'match': [{'input': {'addrKey': '1'}, 'nomina..."
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,"{'match': [{'input': {'addrKey': '2'}, 'nomina..."
3,3,rue du Centre,200,WAIMES,4950,Belgique,"{'match': [{'input': {'addrKey': '3'}, 'nomina..."
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,"{'match': [{'input': {'addrKey': '4'}, 'nomina..."
...,...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique,"{'match': [{'input': {'addrKey': '2995'}, 'nom..."
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique,"{'match': [{'input': {'addrKey': '2996'}, 'nom..."
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique,"{'match': [{'input': {'addrKey': '2997'}, 'nom..."
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique,"{'match': [{'input': {'addrKey': '2998'}, 'nom..."


In [92]:
addresses_seq.json.iloc[784]

In [114]:
addresses_seq_out = format_ws_res(addresses_seq)
addresses_seq_out

input           nominatim                                       work  \
     addrKey                 lat                 lon placeRank       method   
0          0          50.5556305   4.907401297756147        30         fast   
1          1          50.4627801           4.8655064        26         fast   
2          2  50.868676699999995   4.321060219298245        30         fast   
3          3          50.4161219           6.1119552        26         fast   
4          4          51.0399911           4.8089699        30         fast   
...      ...                 ...                 ...       ...          ...   
2995    2995           50.462361  4.5964580703527815        30  regex[init]   
2996    2996          50.6394448           3.3490581        26         fast   
2997    2997          50.4797372           3.8389755        26         fast   
2998    2998          50.6063976           4.1594953        26         fast   
2999    2999          50.4919286   5.866972229816352        30         fast   

                                          output                               \
                                      streetName             city houseNumber   
0                              Chaussée de Namur            Leuze         198   
1                                Rue de la Halle            Namur         NaN   
2     Avenue Jacques Sermon - Jacques Sermonlaan        Ganshoren          38   
3                                  Rue du Centre           Waimes         NaN   
4                             Kleine Waterstraat         Hulshout           9   
...                                          ...              ...         ...   
2995                              Rue du Wainage      Sambreville          62   
2996                         Chaussée de Tournai   Ramegnies-Chin         NaN   
2997                              Rue Louis Caty   Saint-Ghislain         NaN   
2998                          Drève des Bouleaux  Braine-le-Comte         NaN   
2999                                  Rue Rogier              Spa           1   

                                                                           \
                          country postCode inHouseNumber lpostHouseNumber   
0     België / Belgique / Belgien     5310           198              198   
1     België / Belgique / Belgien     5000            11               11   
2     België / Belgique / Belgien     1083            38               38   
3     België / Belgique / Belgien     4950           200              200   
4     België / Belgique / Belgien     2235             9                9   
...                           ...      ...           ...              ...   
2995  België / Belgique / Belgien     5060            62               62   
2996  België / Belgique / Belgien     7520             4                4   
2997  België / Belgique / Belgien     7331            32               32   
2998  België / Belgique / Belgien     7090             5                5   
2999  België / Belgique / Belgien     4900             1                1   

                                 
     lpostUnit            other  
0                           NaN  
1                           NaN  
2                           NaN  
3                           NaN  
4                           NaN  
...        ...              ...  
2995       NaN  Keumiée - Namur  
2996                        NaN  
2997                        NaN  
2998                        NaN  
2999                        NaN  

[3000 rows x 14 columns]

In [115]:
addresses_seq_out[("work", "method")].value_counts(dropna=False)

fast                             2725
regex[init]                       113
libpostal+regex[lpost]+photon      90
libpostal+regex[lpost]             46
nostreet                           19
libpostal+regex[lpost]+nonum        3
nonum                               2
NaN                                 1
photon                              1
Name: (work, method), dtype: int64

In [116]:
# addresses_seq_out[addresses_seq_out[("work", 0)].isnull()]

In [117]:
# addresses_seq_out[addresses_seq.json.apply(lambda js: "osmOrder" in js["match"][0]['work'] if js else None).fillna(False)]

In [118]:
addresses_seq

,addrKey,streetName,houseNumber,city,postCode,country,json
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique,"{'match': [{'input': {'addrKey': '0'}, 'nomina..."
1,1,Rue de la Halle,11,NAMUR,5000,Belgique,"{'match': [{'input': {'addrKey': '1'}, 'nomina..."
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,"{'match': [{'input': {'addrKey': '2'}, 'nomina..."
3,3,rue du Centre,200,WAIMES,4950,Belgique,"{'match': [{'input': {'addrKey': '3'}, 'nomina..."
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,"{'match': [{'input': {'addrKey': '4'}, 'nomina..."
...,...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique,"{'match': [{'input': {'addrKey': '2995'}, 'nom..."
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique,"{'match': [{'input': {'addrKey': '2996'}, 'nom..."
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique,"{'match': [{'input': {'addrKey': '2997'}, 'nom..."
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique,"{'match': [{'input': {'addrKey': '2998'}, 'nom..."


### Using Dask

In [119]:
addresses_dask = addresses.copy()

In [120]:
addresses_dask

,addrKey,streetName,houseNumber,city,postCode,country
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique
1,1,Rue de la Halle,11,NAMUR,5000,Belgique
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique
3,3,rue du Centre,200,WAIMES,4950,Belgique
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique
...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique


In [121]:
t = datetime.now()
dd_addresses = dd.from_pandas(addresses_dask.fillna(""), npartitions=8)

dask_task = dd_addresses.apply(call_ws, check_result=False, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses_dask["json"] = dask_task.compute()
    
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_dask.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 24.52 seconds, 40.79 it/s
# Fastmode:    15.81 seconds, 63.27 it/s


# Resto dataset:
# Normal mode: 27.86 seconds, 35.89 it/s
# Fast mode:   18.44 seconds, 54.23 it/s

# Best dataset: 
# Normal mode: 16.11 seconds, 62.07 it/s
# Fast mode:    9.76 seconds, 102.42 it/s

[                                        ] | 0% Completed |  5.6sNo result!
addrKey                      784
streetName     route de Balmoral
houseNumber                   35
city                SART-LEZ-SPA
postCode                    4845
country                 Belgique
Name: 784, dtype: object
b''
[########################################] | 100% Completed | 35.9s
35.98 seconds, 83.38 it/s


In [27]:
# 1000, 1 worker: 4m18
# 4 workers, npart=4 : 1m20
# 8 workers, npart=4 : 1m20
# 8 workers, npart=8 : 44s

# with checker=False:
# 8 workers, npart=8 : 24s


In [122]:
# expand_json(addresses_dask)

addresses_dask_out = format_ws_res(addresses_dask)
addresses_dask_out

work                                                              \
           method                                 transformedAddress osmOrder   
0            fast       Chaussée de Namur, 198, 5310 LEUZE, Belgique      NaN   
1            fast          Rue de la Halle, 11, 5000 NAMUR, Belgique      NaN   
2            fast  Avenue Jacques Sermon, 38, 1083 GANSHOREN, Bel...      NaN   
3            fast          rue du Centre, 200, 4950 WAIMES, Belgique      NaN   
4            fast     Kleine Waterstraat, 9, 2235 HULSHOUT, Belgique      NaN   
...           ...                                                ...      ...   
2995  regex[init]     Rue du Wainage, 62, 5060 Sambreville, Belgique        0   
2996         fast  Chaussée de Tournai(R-C), 4, 7520 Tournai, Bel...      NaN   
2997         fast  Rue Louis Caty(B), 32, 7331 Saint-Ghislain, Be...      NaN   
2998         fast  Drève des Bouleaux, 5, 7090 Braine-le-Comte, B...      NaN   
2999         fast                  Rue Rogier, 1, 4900 Spa, Belgique      NaN   

                                     \
     cleansedHouseNumber retryOn_26   
0                    NaN        NaN   
1                    NaN        NaN   
2                    NaN        NaN   
3                    NaN        NaN   
4                    NaN        NaN   
...                  ...        ...   
2995                 NaN        NaN   
2996                 NaN        NaN   
2997                 NaN        NaN   
2998                 NaN        NaN   
2999                 NaN        NaN   

                                              nominatim           \
                                            displayName  placeId   
0     198, Chaussée de Namur, Leuze, Éghezée, Namur,...  2580480   
1     Rue de la Halle, Namur, Wallonie, 5000, België...   392307   
2     38, Avenue Jacques Sermon - Jacques Sermonlaan...  1094419   
3     Rue du Centre, Waimes, Verviers, Liège, Wallon...   342040   
4     Tennisclub Lybo, 9, Kleine Waterstraat, Houtve...   171744   
...                                                 ...      ...   
2995  62, Rue du Wainage, Keumiée, Sambreville, Namu...  2657266   
2996  Chaussée de Tournai, Hurette, Ramegnies-Chin, ...  1983533   
2997  Rue Louis Caty, Quartier de la Charbonnière, B...   432987   
2998  Drève des Bouleaux, Coraimont, Braine-le-Comte...   379561   
2999  1, Rue Rogier, Spa, Verviers, Liège, Wallonie,...  1868481   

                                                        ...        output  \
                     lat                 lon placeRank  ... inHouseNumber   
0             50.5556305   4.907401297756147        30  ...           198   
1             50.4627801           4.8655064        26  ...            11   
2     50.868676699999995   4.321060219298245        30  ...            38   
3             50.4161219           6.1119552        26  ...           200   
4             51.0399911           4.8089699        30  ...             9   
...                  ...                 ...       ...  ...           ...   
2995           50.462361  4.5964580703527815        30  ...            62   
2996          50.6394448           3.3490581        26  ...             4   
2997          50.4797372           3.8389755        26  ...            32   
2998          50.6063976           4.1594953        26  ...             5   
2999          50.4919286   5.866972229816352        30  ...             1   

                                                                     input  \
     lpostHouseNumber lpostUnit            other                streetName   
0                 198                        NaN         Chaussée de Namur   
1                  11                        NaN           Rue de la Halle   
2                  38                        NaN     Avenue Jacques Sermon   
3                 200                        NaN             rue du Centre   
4                   9                        NaN        Kleine Waterstraat   
...               

In [123]:
# addresses_dask.method.value_counts()#.json.loc[550]
addr_key_field

'addrKey'

In [124]:
fields = [("input", addr_key_field), ("work", "method"), ("output", city_field), ("output", postcode_field),("output", street_field), ("output", housenbr_field)]
mg = addresses_seq_out[fields].merge(
    addresses_dask_out[fields], how="outer", indicator=True)
if mg.shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


/tmp/ipykernel_39732/3206381806.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  mg = addresses_seq_out[fields].merge(


In [31]:
mg

input         work           output           \
     addrKey       method             city postCode   
0          0         fast            Leuze     5310   
1          1         fast            Namur     5000   
2          2         fast        Ganshoren     1083   
3          3         fast           Waimes     4950   
4          4         fast         Hulshout     2235   
...      ...          ...              ...      ...   
2995    2995  regex[init]      Sambreville     5060   
2996    2996         fast   Ramegnies-Chin     7520   
2997    2997         fast   Saint-Ghislain     7331   
2998    2998         fast  Braine-le-Comte     7090   
2999    2999         fast              Spa     4900   

                                                             _merge  
                                      streetName houseNumber         
0                              Chaussée de Namur         198   both  
1                                Rue de la Halle         NaN   both  
2     Avenue Jacques Sermon - Jacques Sermonlaan          38   both  
3                                  Rue du Centre         NaN   both  
4                             Kleine Waterstraat           9   both  
...                                          ...         ...    ...  
2995                              Rue du Wainage          62   both  
2996                         Chaussée de Tournai         NaN   both  
2997                              Rue Louis Caty         NaN   both  
2998                          Drève des Bouleaux         NaN   both  
2999                                  Rue Rogier           1   both  

[3000 rows x 7 columns]

## Batch calls (batch WS)

### Single block

In [125]:
# addresses=addresses[addresses.addrKey.isin(["1075", "1331", "2616"])]
# addresses=addresses[addresses.addrKey.isin(["2206", "2556", "2569", "2715"])]
# addresses=addresses.iloc[0:100]
addresses

,addrKey,streetName,houseNumber,city,postCode,country
0,0,Chaussée de Namur,198,LEUZE,5310,Belgique
1,1,Rue de la Halle,11,NAMUR,5000,Belgique
2,2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique
3,3,rue du Centre,200,WAIMES,4950,Belgique
4,4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique
...,...,...,...,...,...,...
2995,2995,Rue du Wainage(VEL),62,Sambreville,5060,Belgique
2996,2996,Chaussée de Tournai(R-C),4,Tournai,7520,Belgique
2997,2997,Rue Louis Caty(B),32,Saint-Ghislain,7331,Belgique
2998,2998,Drève des Bouleaux,5,Braine-le-Comte,7090,Belgique


In [126]:
t = datetime.now()

addresses_batch, rejected = call_ws_batch(addresses[[addr_key_field, 
                                           street_field, housenbr_field, postcode_field, city_field, country_field]], 
                                mode="long", 
                                check_result=False, 
                                structured_osm=False,
                                with_rejected=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO dataset: 33.94 seconds, 29.46 it/s
# Best:        24.99 seconds, 40.01 it/s
# Resto:       38.33 seconds, 26.09 it/s

addresses_batch

101.42 seconds, 29.58 it/s


work                     \
                                     transformedAddress osmOrder    method   
0          Chaussée de Namur, 198, 5310 LEUZE, Belgique      0.0      orig   
1             Rue de la Halle, 11, 5000 NAMUR, Belgique      0.0      orig   
2     Avenue Jacques Sermon, 38, 1083 GANSHOREN, Bel...      0.0      orig   
3             rue du Centre, 200, 4950 WAIMES, Belgique      0.0      orig   
4        Kleine Waterstraat, 9, 2235 HULSHOUT, Belgique      0.0      orig   
...                                                 ...      ...       ...   
2994                        9070 Destelbergen, Belgique      0.0  nostreet   
2995                            8550 Zwevegem, Belgique      0.0  nostreet   
2996                            8550 Zwevegem, Belgique      0.0  nostreet   
2997                       6230 Pont-à-Celles, Belgique      0.0  nostreet   
2998                          1770 Liedekerke, Belgique      0.0  nostreet   

                  input                                              \
     retryOn_26 addrKey             streetName houseNumber postCode   
0           NaN       0      Chaussée de Namur         198     5310   
1           NaN       1        Rue de la Halle          11     5000   
2           NaN       2  Avenue Jacques Sermon          38     1083   
3           NaN       3          rue du Centre         200     4950   
4           NaN       4     Kleine Waterstraat           9     2235   
...         ...     ...                    ...         ...      ...   
2994        NaN    1207      Blauwesteenstraat         117     9070   
2995        NaN    1460   Gustave Desmetstraat          15     8550   
2996        NaN    1835         Denemarkenlaan          14     8550   
2997        NaN    2116        Rue Ecluse(PAC)           -     6230   
2998        NaN    2891        NIjverheidslaan          24     1770   

                               ...        nominatim  \
               city   country  ... isolatedDwelling   
0             LEUZE  Belgique  ...              NaN   
1             NAMUR  Belgique  ...              NaN   
2         GANSHOREN  Belgique  ...              NaN   
3            WAIMES  Belgique  ...              NaN   
4          HULSHOUT  Belgique  ...              NaN   
...             ...       ...  ...              ...   
2994   Destelbergen  Belgique  ...              NaN   
2995       Zwevegem  Belgique  ...              NaN   
2996       Zwevegem  Belgique  ...              NaN   
2997  Pont-à-Celles  Belgique  ...              NaN   
2998     Liedekerke  Belgique  ...              NaN   

                                          output                             \
                                      streetName           city houseNumber   
0                              Chaussée de Namur          Leuze         198   
1                                Rue de la Halle          Namur         NaN   
2     Avenue Jacques Sermon - Jacques Sermonlaan      Ganshoren          38   
3                                  Rue du Centre         Waimes         NaN   
4                             Kleine Waterstraat       Hulshout           9   
...                                          ...            ...         ...   
2994                                         NaN   Destelbergen         NaN   
2995                                         NaN       Zwevegem         NaN   
2996                                         NaN       Zwevegem         NaN   
2997                                         NaN  Pont-à-Celles         NaN   
2998                                         NaN     Liedekerke         NaN   

                                            \
                          country postCode   
0     België / Belgique / Belgien     5310   
1     België / Belgique / Belgien     5000   
2     België / Belgique / Belgien     1083   
3     België / Belgique / Belgien     4950   
4     België / Belgique / Belgien     2235   
...                           ...      ...   


In [132]:
addresses_batch[("work", "method")].value_counts()

orig                             2725
regex[init]                       113
libpostal+regex[lpost]+photon      90
libpostal+regex[lpost]             46
nostreet                           19
libpostal+regex[lpost]+nonum        3
nonum                               2
photon                              1
Name: (work, method), dtype: int64

In [127]:
rejected

work                        \
                                  transformedAddress osmOrder rejectReason   
0     Kleine Waterstraat, 9, 2235 HULSHOUT, Belgique        1         tail   
1        Place Jourdan, 20, 1040 ETTERBEEK, Belgique        1         tail   
2        Place Jourdan, 20, 1040 ETTERBEEK, Belgique        2         tail   
3        Place Jourdan, 20, 1040 ETTERBEEK, Belgique        3         tail   
4     rue des Charmilles, 67, 4950 MALMEDY, Belgique        1         tail   
...                                              ...      ...          ...   
1244                           2020 ANVERS, Belgique        1         tail   
1245                    2000 ANVERS-CENTRE, Belgique        1         tail   
1246                           2018 ANVERS, Belgique        1         tail   
1247                          4950 MALMEDY, Belgique        1         tail   
1248                          4950 MALMEDY, Belgique        2         tail   

                 input nominatim                                         \
        method addrKey   placeId                lat                 lon   
0         orig       4   1477173        51.03990125   4.808706157426299   
1         orig       9    318157         50.8375788           4.3814168   
2         orig       9   3997089        50.83710495  4.3812600938545625   
3         orig       9    251739         50.8376505            4.380806   
4         orig      13    843915         50.4610681           6.0684755   
...        ...     ...       ...                ...                 ...   
1244  nostreet     179   4004237  51.18789835865052  4.3838125259454355   
1245  nostreet     337   4004235  51.21931526123043   4.403887342287709   
1246  nostreet     602   4004236  51.20528447247316   4.415824429237694   
1247  nostreet     682    557260         50.5096151            6.060531   
1248  nostreet     682   1479120         50.4151826           6.1052682   

                                                                   ...         \
                                            displayName placeRank  ... suburb   
0     9, Kleine Waterstraat, Houtvenne, Hulshout, Tu...        30  ...    NaN   
1     Place Jourdan - Jourdanplein, Etterbeek, Bruss...        26  ...    NaN   
2     Place Jourdan - Jourdanplein, Etterbeek, Bruss...        26  ...    NaN   
3     Place Jourdan - Jourdanplein, Bruxelles - Brus...        26  ...    NaN   
4     Rue des Charmilles, Xhoffraix, Malmedy, Vervie...        26  ...    NaN   
...                                                 ...       ...  ...    ...   
1244  Antwerpen, Vlaanderen, 2020, België / Belgique...        21  ...    NaN   
1245  Antwerpen, Vlaanderen, 2000, België / Belgique...        21  ...    NaN   
1246  Antwerpen, Vlaanderen, 2018, België / Belgique...        21  ...    NaN   
1247  Ancien Chemin de Malmedy à Eupen, Ovifat, Robe...        26  ...    NaN   
1248  Rue de Malmedy, Waimes, Verviers, Liège, Wallo...        26  ...    NaN   

                                                                   output  \
     municipality farm isolatedDwelling                        streetName   
0             NaN  NaN              NaN                Kleine Waterstraat   
1             NaN  NaN              NaN      Place Jourdan - Jourdanplein   
2             NaN  NaN              NaN      Place Jourdan - Jourdanplein   
3             NaN  NaN              NaN      Place Jourdan - Jourdanplein   
4             NaN  NaN              NaN                Rue des Charmilles   
...           ...  ...              ...                               ...   
1244          NaN  NaN              NaN                               NaN   
1245          NaN  NaN              NaN                               NaN   
1246          NaN  NaN              NaN                               NaN   
1247       Waimes  NaN              NaN  Ancien Chemin de Malmedy à Eupen   
1248       Waimes  NaN              NaN                    Rue de Malmedy   

 

In [128]:
# mg = addresses_seq[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna("").replace("fast", "orig").merge(
#     addresses_batch[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna(""), how="outer", indicator=True)

fields = [("input", addr_key_field), ("work", "method"), ("output", city_field), ("output", postcode_field),("output", street_field), ("output", housenbr_field)]
mg = addresses_seq_out[fields].dropna(subset=[("input", addr_key_field)]).replace("fast", "orig").merge(
    addresses_batch[fields], how="outer", indicator=True)

# mg = addresses_seq[[ "city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna("").replace("fast", "orig").merge(
#      addresses_batch[["city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses_batch.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


/tmp/ipykernel_39732/1728016647.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  mg = addresses_seq_out[fields].dropna(subset=[("input", addr_key_field)]).replace("fast", "orig").merge(


In [129]:
mg

input         work           output           \
     addrKey       method             city postCode   
0          0         orig            Leuze     5310   
1          1         orig            Namur     5000   
2          2         orig        Ganshoren     1083   
3          3         orig           Waimes     4950   
4          4         orig         Hulshout     2235   
...      ...          ...              ...      ...   
2994    2995  regex[init]      Sambreville     5060   
2995    2996         orig   Ramegnies-Chin     7520   
2996    2997         orig   Saint-Ghislain     7331   
2997    2998         orig  Braine-le-Comte     7090   
2998    2999         orig              Spa     4900   

                                                             _merge  
                                      streetName houseNumber         
0                              Chaussée de Namur         198   both  
1                                Rue de la Halle         NaN   both  
2     Avenue Jacques Sermon - Jacques Sermonlaan          38   both  
3                                  Rue du Centre         NaN   both  
4                             Kleine Waterstraat           9   both  
...                                          ...         ...    ...  
2994                              Rue du Wainage          62   both  
2995                         Chaussée de Tournai         NaN   both  
2996                              Rue Louis Caty         NaN   both  
2997                          Drève des Bouleaux         NaN   both  
2998                                  Rue Rogier           1   both  

[2999 rows x 7 columns]

In [130]:
# Geocode + address
match, rej = call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="geo", check_result=False, with_rejected=True) 
match

input           nominatim                                    work
     addrKey                 lat                 lon placeRank    method
0          0          50.5556305   4.907401297756147        30      orig
1          1          50.4627801           4.8655064        26      orig
2          2  50.868676699999995   4.321060219298245        30      orig
3          3          50.4161219           6.1119552        26      orig
4          4          51.0399911           4.8089699        30      orig
...      ...                 ...                 ...       ...       ...
2994    1207   51.05701251731331    3.79924146285496        21  nostreet
2995    1460  50.816471197699684   3.339828310552459        21  nostreet
2996    1835  50.816471197699684   3.339828310552459        21  nostreet
2997    2116  50.511907912027056  4.3675044878988505        21  nostreet
2998    2891   50.87537991500001   4.097055137283286        21  nostreet

[2999 rows x 5 columns]

In [39]:
# Geocode + address
match, rej = call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="short", check_result=False) 
match


input           nominatim                                         \
     addrKey                 lat                 lon placeRank  placeId   
0          0          50.5556305   4.907401297756147        30  2580480   
1          1          50.4627801           4.8655064        26   392307   
2          2  50.868676699999995   4.321060219298245        30  1094419   
3          3          50.4161219           6.1119552        26   342040   
4          4          51.0399911           4.8089699        30   171744   
...      ...                 ...                 ...       ...      ...   
2994    2995           50.462361  4.5964580703527815        30  2657266   
2995    2996          50.6394448           3.3490581        26  1983533   
2996    2997          50.4797372           3.8389755        26   432987   
2997    2998          50.6063976           4.1594953        26   379561   
2998    2999          50.4919286   5.866972229816352        30  1868481   

             work                                      output  \
           method                                  streetName   
0            orig                           Chaussée de Namur   
1            orig                             Rue de la Halle   
2            orig  Avenue Jacques Sermon - Jacques Sermonlaan   
3            orig                               Rue du Centre   
4            orig                          Kleine Waterstraat   
...           ...                                         ...   
2994  regex[init]                              Rue du Wainage   
2995         orig                         Chaussée de Tournai   
2996         orig                              Rue Louis Caty   
2997         orig                          Drève des Bouleaux   
2998         orig                                  Rue Rogier   

                                                                         \
                 city houseNumber                      country postCode   
0               Leuze         198  België / Belgique / Belgien     5310   
1               Namur         NaN  België / Belgique / Belgien     5000   
2           Ganshoren          38  België / Belgique / Belgien     1083   
3              Waimes         NaN  België / Belgique / Belgien     4950   
4            Hulshout           9  België / Belgique / Belgien     2235   
...               ...         ...                          ...      ...   
2994      Sambreville          62  België / Belgique / Belgien     5060   
2995   Ramegnies-Chin         NaN  België / Belgique / Belgien     7520   
2996   Saint-Ghislain         NaN  België / Belgique / Belgien     7331   
2997  Braine-le-Comte         NaN  België / Belgique / Belgien     7090   
2998              Spa           1  België / Belgique / Belgien     4900   

                                                                    \
                                                  other inHouseNbr   
0                                       Namur - Éghezée        198   
1                                                   NaN         11   
2                Brussel-Hoofdstad - Bruxelles-Capitale         38   
3                                      Liège - Verviers        200   
4     Tennisclub Lybo - Antwerpen - Houtvenne - Turn...          9   
...                                                 ...        ...   
2994                                    Keumiée - Namur         62   
2995     Hurette - Tournai - Tournai-Mouscron - Hainaut          4   
2996  Mons - Quartier de la Charbonnière - Hainaut -...         32   
2997                     Coraimont - Soignies - Hainaut          5   
2998                                   Liège - Verviers          1   

                              
     lpostHouseNbr lpostUnit  
0              198       NaN  
1               11       NaN  
2               38       NaN  
3              200       NaN  
4                9       NaN  
...            ...       ...  
2994            62       NaN  
2995             4 

In [40]:
# addresses_batch.columns

In [41]:
# Geocode + address, with rejected addresses
match, rej = call_ws_batch(addresses, mode="long", with_rejected=True)
match

work                     \
                                     transformedAddress osmOrder    method   
0          Chaussée de Namur, 198, 5310 LEUZE, Belgique      0.0      orig   
1             Rue de la Halle, 11, 5000 NAMUR, Belgique      0.0      orig   
2                Steenweg, 269, 9810 Nazareth, Belgique      0.0      orig   
3              Centrumstraat, 5, 3012 Louvain, Belgique      0.0      orig   
4     Rue Félix De Cuyper, 39, 1070 Anderlecht, Belg...      0.0      orig   
...                                                 ...      ...       ...   
2994                         2070 ZWIJNDRECHT, Belgique      0.0  nostreet   
2995                             4950 MALMEDY, Belgique      0.0  nostreet   
2996                           5580 ROCHEFORT, Belgique      0.0  nostreet   
2997                   5537 ANNEVOIE-ROUILLON, Belgique      0.0  nostreet   
2998                              2030 ANVERS, Belgique      0.0  nostreet   

                  input                                                     \
     retryOn_26 addrKey                                         streetName   
0           NaN       0                                  Chaussée de Namur   
1           NaN       1                                    Rue de la Halle   
2           NaN    1000                                           Steenweg   
3           NaN    1001                                      Centrumstraat   
4           NaN    1002                                Rue Félix De Cuyper   
...         ...     ...                                                ...   
2994        NaN     669                                     Stationsstraat   
2995        NaN     682                          Rue du Lac | Robertville,   
2996        NaN     758  Rue du Sourd d'Ave | situé au confins de la N9...   
2997        NaN      93                                  route des Jardins   
2998        NaN      94           Siberiastraat Z/N (voormalig droogdok 7)   

                                                        ...   output  \
     houseNumber               city postCode   country  ... postCode   
0            198              LEUZE     5310  Belgique  ...     5310   
1             11              NAMUR     5000  Belgique  ...     5000   
2            269           Nazareth     9810  Belgique  ...     9810   
3              5            Louvain     3012  Belgique  ...     3012   
4             39         Anderlecht     1070  Belgique  ...     1070   
...          ...                ...      ...       ...  ...      ...   
2994          93        ZWIJNDRECHT     2070  Belgique  ...     2070   
2995          24            MALMEDY     4950  Belgique  ...     4960   
2996           5          ROCHEFORT     5580  Belgique  ...     5580   
2997          35  ANNEVOIE-ROUILLON     5537  Belgique  ...     5537   
2998        [na]             ANVERS     2030  Belgique  ...     2030   

                                                                    \
                                                  other inHouseNbr   
0                                       Namur - Éghezée        198   
1                                                   NaN         11   
2                                Gent - Oost-Vlaanderen        269   
3                               Leuven - Vlaams-Brabant          5   
4     Aa - Veeweyde - Veeweide - Brussel-Hoofdstad -...         39   
...                                                 ...        ...   
2994                                          Antwerpen         93   
2995                                   Liège - Verviers         24   
2996                                     Namur - Dinant          5   
2997                                     Namur - Dinant         35   
2998                                          Antwerpen       [na]   

                                 check                                    \
     lpostHouseNbr lpostUnit SIMStreet SIMStreetWhich SIMHouseNbr SIMZip   
0              198   

### Batch blocs

In [133]:
# addresses = addresses.sample(10000, replace=True)
# addresses = addresses.reset_index(drop=True)
# addresses["addr_key"]= addresses.index.astype(str)

In [134]:
t = datetime.now()

nb_threads=8

chunks = np.array_split(addresses, nb_threads) # addresses.shape[0]//100)

print(f"{len(chunks)} chunks on {nb_threads} threads")

import multiprocess as mp

p = mp.Pool(nb_threads)

def f(chunk):
    return call_ws_batch(chunk, mode="long", 
                        check_result=False, 
                        structured_osm=False)[0]

with p:
     res= list(tqdm(p.imap(f, chunks), total=len(chunks)))
    
addresses_batch2 = pd.concat(res).reset_index(drop=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO:    9.28 seconds, 107.72 it/s
# Best:   6.88 seconds, 145.43 it/s
# Resto: 11.79 seconds,  84.85 it/s

8 chunks on 8 threads


  0%|          | 0/8 [00:00<?, ?it/s]

23.78 seconds, 126.17 it/s


In [44]:
# addresses_batch2
# expand_batch_address(addresses_batch2)

In [135]:
fields = [("input", addr_key_field), ("work", "method"), ("output", city_field), ("output", postcode_field),("output", street_field), ("output", housenbr_field)]
mg = addresses_seq_out[fields].dropna(subset=[("input", addr_key_field)]).replace("fast", "orig").merge(
    addresses_batch2[fields], how="outer", indicator=True)


# mg = addresses_seq[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna("").replace("fast", "orig").merge(
#      addresses_batch2[[addr_key_field, city_field, postcode_field,street_field, housenbr_field, f"addrOut{street_field}", f"addrOut{housenbr_field}", f"addrOut{postcode_field}", f"addrOut{city_field}"]].fillna(""), how="outer", indicator=True)

# mg = addresses_seq[[  "city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna("").replace("fast", "orig").merge(
#     addresses_batch2[["city", "postcode","street", "housenumber", "method", "addrOutStreet", "addrOutNumber", "addrOutPostcode", "addrOutCity", "addrKey"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses_batch2.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


/tmp/ipykernel_39732/2124162270.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  mg = addresses_seq_out[fields].dropna(subset=[("input", addr_key_field)]).replace("fast", "orig").merge(


## Comparing options

In [46]:
# addresses = get_addresses("address.csv.gz")
# addresses = addresses[addresses[country_field] == "Belgique"]
# # addresses = addresses.sample(10000).copy()

In [136]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        #expand_batch_address(results[(check_label, struct_label)])
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

check struct
check unstruct
nocheck struct
nocheck unstruct
Iterations per seconds:


,struct,unstruct
check,34.569840,29.894258
nocheck,39.312862,32.324878


In [137]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)][0].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate


,check,nocheck
struct,1.000000,1.000000
unstruct,0.999667,0.999667


In [138]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)][0][results[(k1,k2)][0][('work', 'method')]!='nostreet'].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate (without nostreet)


,check,nocheck
struct,0.990333,0.992667
unstruct,0.990667,0.993333


In [139]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1][0][("input", "addrKey")])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

Unmatched addresses
('check', 'struct')


,addrKey,streetName,houseNumber,city,postCode,country


Series([], Name: country, dtype: int64)
('check', 'unstruct')


,addrKey,streetName,houseNumber,city,postCode,country
784,784,route de Balmoral,35,SART-LEZ-SPA,4845,Belgique


Belgique    1
Name: country, dtype: int64
('nocheck', 'struct')


,addrKey,streetName,houseNumber,city,postCode,country


Series([], Name: country, dtype: int64)
('nocheck', 'unstruct')


,addrKey,streetName,houseNumber,city,postCode,country
784,784,route de Balmoral,35,SART-LEZ-SPA,4845,Belgique


Belgique    1
Name: country, dtype: int64
